In [52]:
import os
from dotenv import load_dotenv
load_dotenv()   

from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")  # ✅ Correct spelling

model= ChatGroq(
    model_name="gemma2-9b-it",groq_api_key=groq_api_key)

model


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002368BEBFC80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002368BF34320>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [30]:
ollama_api_key = os.getenv("OLLAMA_API_KEY")
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings( model="mxbai-embed-large")


In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


In [21]:
import bs4
loader=WebBaseLoader(web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content","post-content-inner","post-content-inner-wrapper","post-title"))))

In [23]:
docs=loader.load()

In [31]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000236857BFDD0>, search_kwargs={})

In [32]:
system_prompt=("you are a helpful assistant that answers questions based on the provided context. If the context does not contain the answer, you should say 'I don't know'.use three sentences to answer the question. If the context is not enough, you should say 'I don't know' and ask the user to provide more information.""\n\n""{context}")


prompt = ChatPromptTemplate.from_messages([("system",system_prompt), ("human", "{input}")])

In [53]:
question_answer_chain = create_stuff_documents_chain(model,prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [59]:
response = rag_chain.invoke({"input": "What is the self reflection?"})
print(response["answer"])


Self-reflection, in the context of this document, is the process of an agent analyzing past actions and identifying areas for improvement. 

This analysis is used to guide future decision-making and correct past mistakes. 

It's achieved by showing the agent examples of failed trajectories paired with ideal reflections that suggest changes to the plan. 






In [76]:
##Adding the chat history to the chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and a question, provide a context for the question based on the chat history. "
    "If the chat history does not contain enough information, say 'I don't know' and ask the user to provide more information.\n\n"
    "Chat History:\n{chat_history}\n\nQuestion: {input}\n\nContext:"
)

contextualize_q_prompt= ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),])




In [77]:
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),])

In [78]:
history_aware_retriever= create_history_aware_retriever(model, retriever, contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000236857BFDD0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChun

In [79]:
question_answer_chain = create_stuff_documents_chain(model,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [83]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []
question= "What is the self reflection?"
response1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response1["answer"])
])

question2="Tell me more about it?"
response2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question2),
    AIMessage(content=response2["answer"])])
# print(response2["answer"])

question3="Why using that?"
response3 = rag_chain.invoke({"input": question3, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question3),
    AIMessage(content=response3["answer"])])
print(response3["answer"])




Using self-reflection is beneficial because it enables autonomous agents to learn and improve iteratively. 

By analyzing past actions and identifying failures, agents can generate insights that lead to better decision-making in the future.  

This ability to learn from experience is crucial for agents operating in complex and dynamic environments where trial and error are inevitable. 

